DeeplabV3+

In [1]:
import os
import json
import logging
import copy
import shutil
import time

from tqdm import tqdm

import cv2
import numpy as np
from PIL import Image

import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist

from torchvision import transforms
from torchvision.transforms import functional as F

from math import ceil
from itertools import cycle
from itertools import chain
from collections import OrderedDict
from functools import partial

import matplotlib.pyplot as plt
import seaborn as sns

import random
random.seed(42)

/home/cvrlab308/anaconda3/envs/ps-mt/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#config
batch_size = 8
epochs = 80
warm_up = 80
labeled_examples = 515
lr = 1e-2
backbone = 101 #" the resnet x {50, 101} layers"
semi_p_th = 0.6 # positive_threshold for semi-supervised loss
semi_n_th = 0.0 # negative_threshold for semi-supervised loss
unsup_weight = 1.5 # unsupervised weight for the semi-supervised loss

config = json.load(open("configs/config_deeplab_v3+.json"))

config['train_supervised']['batch_size'] = batch_size
config['train_unsupervised']['batch_size'] = batch_size
config['model']['epochs'] = epochs
config['model']['warm_up_epoch'] = warm_up
config['n_labeled_examples'] = labeled_examples
config['model']['resnet'] = backbone
config['optimizer']['args']['lr'] = lr
config['unsupervised_w'] = unsup_weight
config['model']['data_h_w'] = [config['train_supervised']['crop_size'], config['train_supervised']['crop_size']]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

logger = logging.getLogger("PS-MT")
logger.propagate = False
logger.warning("Training start, 总共 {} epochs".format(str(config['model']['epochs'])))
logger.critical("GPUs: {}".format(device))
logger.critical("DeeplabV3+ with ResNet {} backbone".format(str(config['model']['resnet'])))
logger.critical("Current Labeled Example: {}".format(config['n_labeled_examples']))
logger.critical("Learning rate: other {}, and head is the SAME [world]".format(config['optimizer']['args']['lr']))

logger.critical("Current batch: {} [world]".format(int(config['train_unsupervised']['batch_size']) +
                                                int(config['train_supervised']['batch_size'])) )

logger.critical("Current unsupervised loss function: {}, with weight {} and length {}".format(config['model']['un_loss'],
                                                                                            config['unsupervised_w'],
                                                                                            config['ramp_up']))
print("\nconfig json :")
for i in config:
    print(i, config[i])


Training start, 总共 80 epochs
GPUs: cuda
DeeplabV3+ with ResNet 101 backbone
Current Labeled Example: 515
Learning rate: other 0.01, and head is the SAME [world]
Current batch: 16 [world]
Current unsupervised loss function: semi_ce, with weight 1.5 and length 12



config json :
name PS-MT(DeeplabV3+)
experim_name TEST_warm
n_labeled_examples 515
ramp_up 12
unsupervised_w 1.5
lr_scheduler Poly
gamma 0.5
model {'supervised': False, 'semi': True, 'resnet': 101, 'sup_loss': 'DE', 'un_loss': 'semi_ce', 'epochs': 80, 'warm_up_epoch': 80, 'data_h_w': [224, 224]}
optimizer {'type': 'SGD', 'args': {'lr': 0.01, 'weight_decay': 0.0001, 'momentum': 0.9}}
train_supervised {'data_dir': 'FA', 'batch_size': 8, 'shuffle': True, 'crop_size': 224, 'split': 'train_supervised', 'num_workers': 8}
train_unsupervised {'data_dir': 'unlabel_FA', 'batch_size': 8, 'shuffle': True, 'crop_size': 224, 'split': 'train_unsupervised', 'num_workers': 8}
val_loader {'data_dir': 'FA', 'batch_size': 1, 'split': 'val', 'shuffle': False, 'num_workers': 4}
test_loader {'data_dir': 'FA', 'batch_size': 1, 'split': 'test', 'shuffle': False, 'num_workers': 4}


In [3]:
# DATA LOADERS
from DataLoader.dataset_onlyFA import *
choose_data = "All"

config['train_supervised']['choose'] = choose_data
config['train_unsupervised']['choose'] = choose_data
config['val_loader']['choose'] = choose_data
config['test_loader']['choose'] = choose_data

print("train_supervised")
for i in config['train_supervised']:
    print("    ",i, ":", config['train_supervised'][i])
print("train_unsupervised")
for i in config['train_unsupervised']:
    print("    ", i, ":", config['train_unsupervised'][i])
print("val_loader")
for i in config['val_loader']:
    print("    ", i, ":", config['val_loader'][i])
print("test_loader")
for i in config['test_loader']:
    print("    ", i, ":", config['test_loader'][i])

supervised_set = BasicDataset(data_dir=config['train_supervised']['data_dir'], 
                                 choose=config['train_supervised']['choose'],
                                 split=config['train_supervised']['split'])
unsupervised_set = BasicDataset(data_dir=config['train_unsupervised']['data_dir'],
                                   choose=config['train_unsupervised']['choose'],
                                   split=config['train_unsupervised']['split'])
val_set = BasicDataset(data_dir=config['val_loader']['data_dir'],
                          choose=config['val_loader']['choose'],
                          split=config['val_loader']['split'])

test_set = BasicDataset(data_dir=config['test_loader']['data_dir'],
                          choose=config['test_loader']['choose'],
                          split=config['test_loader']['split'])

print("supervised_loader: ",len(supervised_set))
print("unsupervised_loader: ",len(unsupervised_set))
print("val_loader: ",len(val_set))
print("test_loader: ",len(test_set))


supervised_loader = DataLoader(dataset=supervised_set, batch_size=config['train_supervised']['batch_size'],
                               shuffle=config['train_supervised']['shuffle'], 
                               num_workers=config['train_supervised']['num_workers'])
unsupervised_loader = DataLoader(dataset=unsupervised_set, batch_size=config['train_unsupervised']['batch_size'],
                               shuffle=config['train_unsupervised']['shuffle'], 
                               num_workers=config['train_unsupervised']['num_workers'])
val_loader = DataLoader(dataset=val_set, batch_size=config['val_loader']['batch_size'],
                               shuffle=config['val_loader']['shuffle'], 
                               num_workers=config['val_loader']['num_workers'])
test_loader = DataLoader(dataset=test_set, batch_size=config['test_loader']['batch_size'],
                               shuffle=config['test_loader']['shuffle'], 
                               num_workers=config['test_loader']['num_workers'])



train_supervised
     data_dir : FA
     batch_size : 8
     shuffle : True
     crop_size : 224
     split : train_supervised
     num_workers : 8
     choose : All
train_unsupervised
     data_dir : unlabel_FA
     batch_size : 8
     shuffle : True
     crop_size : 224
     split : train_unsupervised
     num_workers : 8
     choose : All
val_loader
     data_dir : FA
     batch_size : 1
     split : val
     shuffle : False
     num_workers : 4
     choose : All
test_loader
     data_dir : FA
     batch_size : 1
     split : test
     shuffle : False
     num_workers : 4
     choose : All
supervised_loader:  515
unsupervised_loader:  5263
val_loader:  162
test_loader:  157


In [5]:
from torch import optim

from Model.Deeplabv3_plus.psmt_model import *
from Utils.ramps import *


model_t1 = Teacher_Net(num_classes=2, config=config['model'])
model_t1 = model_t1.to(device)

optimizer_t1 = optim.SGD(model_t1.parameters(), 
                      lr=config['optimizer']['args']['lr'],
                      momentum=config['optimizer']['args']['momentum'],
                      weight_decay=config['optimizer']['args']['weight_decay'])

total = sum([param.nelement() for param in model_t1.parameters()])
print("Number of parameter: %.2fM" % (total/1e6))

Number of parameter: 60.75M


In [6]:
from sklearn import metrics, neighbors
from sklearn.metrics import confusion_matrix

def information_index(outputs, targets):
    eps = np.finfo(np.float64).eps
    output = outputs.flatten()
    target = targets.flatten()
    TN, FP, FN, TP = confusion_matrix(target,output).ravel()

    index_MIou =  ( TP / (TP + FP + FN + eps) + TN / (TN + FN + FP + eps) ) / 2
    mean_iou = np.mean(index_MIou)
    index_dice = 2*TP / (2*TP + FP + FN + eps)
    mean_dice = np.mean(index_dice)

    return mean_iou, mean_dice

def count_index(pre, tar):
        path_pre = pre
        path_target = tar
        dirs = os.listdir(path_pre)
        # print(len(dirs))
        con_mIOU = 0
        con_mdice = 0
        for imgs in dirs:
            pre_path = path_pre + '/' + str(imgs)
            target_path = path_target + '/' + str(imgs)

            target = cv2.imread(target_path, cv2.IMREAD_GRAYSCALE)
            _, tar = cv2.threshold(target, 128, 255, cv2.THRESH_BINARY)

            predict = cv2.imread(pre_path, cv2.IMREAD_GRAYSCALE)
            _, pre = cv2.threshold(predict, 128, 255, cv2.THRESH_BINARY)

            tIOU, tdice = information_index(pre,tar)
            con_mIOU += tIOU
            con_mdice += tdice
        val_mIoU = con_mIOU/len(dirs)
        val_mDice = con_mdice/len(dirs)
        
        return val_mIoU, val_mDice

In [6]:
# warm
from Utils.early_stop import EarlyStopping

best_model_t1_params = copy.deepcopy(model_t1.state_dict())
do_best_Dice = 0
do_best_mIoU = 0
do_best_epoch = 0

for epoch in range(config['model']['warm_up_epoch']):

    epoch_loss_t1 = 0

    t1_mIoU, t1_mDice = 0, 0

    
    localtime = time.asctime( time.localtime(time.time()) )
    print('Epoch: {}/{} --- < Starting Time : {} >'.format(epoch + 1,config['model']['warm_up_epoch'],localtime))
    print('-' * len('Epoch: {}/{} --- < Starting Time : {} >'.format(epoch + 1,config['model']['warm_up_epoch'],localtime)))

    folder_name = None
    folder_name = os.path.join("see_image")
    os.makedirs(folder_name, exist_ok=True)

    for batch in tqdm(supervised_loader):
        image_FA, image_ICG, target_l, id_l = batch
        image_FA, image_ICG, target_l, id_l = image_FA.to(device), image_ICG.to(device), target_l.to(device), id_l

        # warm teacher 1
        model_t1.train()
        optimizer_t1.zero_grad()
        loss_t1, outputs = model_t1(x_FA=image_FA, x_ICG=image_ICG, target_l=target_l,
                           warm_up=True,mix_up=False)

        epoch_loss_t1 += loss_t1
        loss_t1.backward()
        optimizer_t1.step()

    print(f'Epoch{epoch+1} loss : \nteacher 1 loss = {epoch_loss_t1/len(supervised_set)}')

    # valiation
    model_t1.eval()
    for batch in tqdm(val_loader):
        image_val, label, id_val = batch
        image_val, label, id_val = image_val.to(device), label.to(device), id_val

        H, W = label.size(1), label.size(2)
        up_sizes = (ceil(H / 8) * 8, ceil(W / 8) * 8)

        for i in range(0, int(label.size(0))):
            folder_val = os.path.join(folder_name, "val_target")
            os.makedirs(folder_val, exist_ok=True)
            image = Image.fromarray(np.uint8(label[i].detach().cpu().numpy()))
            image.save(os.path.join(folder_val, str(id_val[i]) + ".png"))

        data = torch.nn.functional.interpolate(image_val, size=(up_sizes[0], up_sizes[1]),
                                               mode='bilinear', align_corners=True)
        loss_t1, outputs = model_t1(x_FA=data, x_ICG=data, target_l=label,
                                    warm_up=True,mix_up=False)
        output = outputs["sup_pred"]

        for i in range(0, int(output.size(0))):
            folder_val_prob = os.path.join(folder_name, "val_prob")
            os.makedirs(folder_val_prob, exist_ok=True)
            image_prob = output[i].squeeze().detach()
            image_prob = torch.argmax(image_prob, dim=0).cpu().numpy()
            image_prob = Image.fromarray((image_prob * 255).astype(np.uint8))
            image_prob.save(os.path.join(folder_val_prob, str(id_val[i]) + ".png"))
    
    # show epoch mIoU, mDice
    index_mIoU, index_mDice = count_index(folder_val_prob, folder_val)
    print(f'Epoch {epoch+1}' + " val:" + f'mIoU: {index_mIoU:.4f}, DSC: {index_mDice:.4f}')

    # find the best mIoU, mDice
    if index_mDice > do_best_Dice:
        do_best_Dice = index_mDice
        do_best_mIoU = index_mIoU
        do_best_epoch = epoch+1
        best_model_t1_params = copy.deepcopy(model_t1.state_dict())
        print("Change Best: " + f'epoch: {do_best_epoch}, mIoU: {do_best_mIoU:.4f}, DSC: {do_best_Dice:.4f}]')

        # save the best valiation prod
        folder_val_best_prob = os.path.join(folder_name, "val_original_best_prob")
        os.makedirs(folder_val_best_prob, exist_ok=True)
        file_names = os.listdir(folder_val_prob)
        for file_name in file_names:
            source_path = os.path.join(folder_val_prob, file_name)
            destination_path = os.path.join(folder_val_best_prob, file_name)
            shutil.copyfile(source_path, destination_path)
    
    # show the best mIoU, mDice
    print("Best model : " + f'epoch : {do_best_epoch}  DSC : {do_best_Dice:.4f}')

model_path = None
model_path = os.path.join("saved_models")
os.makedirs(model_path, exist_ok=True)
torch.save(best_model_t1_params, os.path.join(model_path, f'original_epoch_{do_best_epoch}_dsc_{do_best_Dice:.4f}_best_t1.pth'))



Epoch: 1/80 --- < Starting Time : Mon Jul  1 09:06:05 2024 >
------------------------------------------------------------


  0%|          | 0/65 [00:00<?, ?it/s]

100%|██████████| 65/65 [00:04<00:00, 14.42it/s]


Epoch1 loss : 
teacher 1 loss = 0.04257368668913841


100%|██████████| 162/162 [00:02<00:00, 70.22it/s]


...

Epoch 79 val:mIoU: 0.5867, DSC: 0.3329
Best model : epoch : 12  DSC : 0.3631
Epoch: 80/80 --- < Starting Time : Mon Jul  1 09:16:59 2024 >
-------------------------------------------------------------


100%|██████████| 65/65 [00:04<00:00, 15.57it/s]


Epoch80 loss : 
teacher 1 loss = 0.001781016355380416


100%|██████████| 162/162 [00:02<00:00, 69.08it/s]


Epoch 80 val:mIoU: 0.5842, DSC: 0.3246
Best model : epoch : 12  DSC : 0.3631


In [7]:
# PRED_MODEL_t1 = os.path.join(model_path, f'original_epoch_{do_best_epoch}_dsc_{do_best_Dice:.4f}_best_t1.pth')
PRED_MODEL_t1 = './saved_models/deeplabV3/original_epoch_5_dsc_0.3550_best_t1.pth'

model_t1.load_state_dict(torch.load(PRED_MODEL_t1, map_location=device))

folder_name = None
folder_name = os.path.join("see_image")
os.makedirs(folder_name, exist_ok=True)

# Test
model_t1.eval()
for batch in tqdm(test_loader):
    image_test, label, id_test = batch
    image_test, label, id_test = image_test.to(device), label.to(device), id_test

    H, W = label.size(1), label.size(2)
    up_sizes = (ceil(H / 8) * 8, ceil(W / 8) * 8)

    for i in range(0, int(label.size(0))):
        folder_test = os.path.join(folder_name, "test_target")
        os.makedirs(folder_test, exist_ok=True)
        image = Image.fromarray(np.uint8(label[i].detach().cpu().numpy()))
        image.save(os.path.join(folder_test, str(id_test[i]) + ".png"))

    data = torch.nn.functional.interpolate(image_test, size=(up_sizes[0], up_sizes[1]),
                                               mode='bilinear', align_corners=True)
    loss_t1, outputs = model_t1(x_FA=data, x_ICG=data, target_l=label,
                                warm_up=True,mix_up=False)
    output = outputs["sup_pred"]

    for i in range(0, int(output.size(0))):
        folder_test_prob = os.path.join(folder_name, "test_prob")
        os.makedirs(folder_test_prob, exist_ok=True)
        image_prob = output[i].squeeze().detach()
        image_prob = torch.argmax(image_prob, dim=0).cpu().numpy()
        image_prob = Image.fromarray((image_prob * 255).astype(np.uint8))
        image_prob.save(os.path.join(folder_test_prob, str(id_test[i]) + ".png"))

# show epoch mIoU, mDice
index_mIoU, index_mDice = count_index(folder_test_prob, folder_test)
print(f'Test  original : mIoU: {index_mIoU:.4f}, DSC: {index_mDice:.4f}')   

100%|██████████| 157/157 [00:02<00:00, 67.37it/s]


Test  original : mIoU: 0.5811, DSC: 0.3340
